In [1]:
import numpy as np
import pandas as pd

from pyaw.swarm import rotate_vector_by_quaternion

In [2]:
fps = ['./SW_OPER_MAGB_HR_1B_1349_20140219T020540_20140219T034014.pkl',
       './IGRF_SW_OPER_MAGB_HR_1B_1349_20140219T020540_20140219T034014.pkl',
       './SW_EXPT_EFIB_TCT16_1349_20140219T020540_20140219T034014.pkl'
       ]
df_b = pd.read_pickle(fps[0])
df_b_igrf = pd.read_pickle(fps[1])
df_e = pd.read_pickle(fps[2])

In [3]:
df_b_clip = df_b[['B_NEC','q_NEC_CRF']].iloc[:600]
df_b_igrf_clip = df_b_igrf[['B_NEC_IGRF']].iloc[:600]
df_e_clip = df_e[['Bx','By','Bz']].iloc[:160]

In [4]:
b_sc = []  # b in S/C
b_igrf_sc = []  # igrf b in S/C
for b_nec,b_nec_igrf,q_NEC_CRF in zip(df_b_clip['B_NEC'].values,df_b_igrf_clip['B_NEC_IGRF'].values,df_b_clip['q_NEC_CRF'].values):
    q_CRF_NEC = np.array([-q_NEC_CRF[0],-q_NEC_CRF[1],-q_NEC_CRF[2],q_NEC_CRF[3]])
    b_sc.append(rotate_vector_by_quaternion(b_nec, q_CRF_NEC))
    b_igrf_sc.append(rotate_vector_by_quaternion(b_nec_igrf, q_CRF_NEC))
b_sc = np.array(b_sc)
b_igrf_sc = np.array(b_igrf_sc)

# b_sc 和 df_e Bx,y,z 有区别，x,z区别达到1e2，y区别达到1e3
# b_igrf_sc 和 df_e Bx,y,z 的区别类似
# 不看单个分量，而是3个分量求得的模（假设都为直角坐标系），区别依旧明显。
# tct 16 (02) 中的bx,y,z是通过1hz产品推导出来的。这个1hz产品是？上述的区别是否是推导产生的？